# 第10章 要件定義書生成AIエージェントの開発

## 設定

In [1]:
import json
import os
import time

from dotenv import load_dotenv
dotenv_path = "../.env"
load_dotenv(dotenv_path)

True

In [2]:
!pip install langchain-core==0.3.0
!pip install langchain-openai==0.2.0
!pip install langgraph==0.2.22
!pip install grandalf


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## データ構造定義

In [3]:
from langchain_core.pydantic_v1 import BaseModel, Field

C:\Users\MikAn\Documents\Python\agent_book\Lib\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
# ペルソナを表すデータモデル
class Persona(BaseModel):
    name: str = Field(
        ...,
        description="ペルソナの名前",
    )
    background: str = Field(
        ...,
        description="ペルソナの持つ背景",
    )

In [5]:
# ペルソナのリストを表すデータモデル
class Personas(BaseModel):
    personas: list[Persona] = Field(
        default_factory=list, # Listをフィールドに定義するときに必要
        description="ペルソナのリスト",
    )

In [6]:
# インタビュー内容を表すデータモデル
class Interview(BaseModel):
    persona: Persona = Field(
        ...,
        description="インタビュー対象のペルソナ",
    )
    question: str = Field(
        ...,
        description="インタビューでの質問"
    )
    answer: str = Field(
        ...,
        description="インタビューの回答"
    )

In [7]:
# インタビュー結果のリストを表すデータモデル
class InterviewResult(BaseModel):
    interviews: list[Interview] = Field(
        default_factory=list,
        description="インタビュー結果のリスト",
    )

In [8]:
# インタビュー結果の評価を表すデータモデル
class EvaluationResult(BaseModel):
    reason: str = Field(
        ...,
        description="判断の理由",
    )
    is_sufficient: bool = Field(
        ...,
        description="情報が十分に足りているか",
    )

## stateの定義

In [9]:
import operator
from typing import Annotated

In [10]:
class InterviewState(BaseModel):
    user_request: str = Field(
        ...,
        description="ユーザからのリクエスト",
    )
    personas: Annotated[list[Persona], operator.add] = Field(
        default=[],
        description="生成されたペルソナのリスト",
    )
    interviews: Annotated[list[Interview], operator.add] = Field(
        default=[],
        description="実施されたインタビュー結果のリスト",
    )
    requirements_doc: str = Field(
        default="",
        description="生成された要件定義",
    )
    iteration: int = Field(
        default=0,
        description="ペルソナ生成とインタビューの反復回数",
    )
    is_information_sufficient: bool = Field(
        default=False,
        description="情報が十分に足りているか",
    )
    evaluation_reason: str =Field(
        default="",
        description="情報が十分に足りているか判断した理由"
    )

## nodeの実処理定義

In [11]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [12]:
# ペルソナ生成
class PersonaGenerator:
    def __init__(self, model: ChatOpenAI, k: int = 5):
        self.model = model.with_structured_output(Personas)
        self.k = k

    def run(self, user_request: str) -> Personas:
        prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "あなたはユーザインタビュー用の多様なペルソナを作成する専門家です。",
                ),
                (
                    "human",
                    "以下のユーザリクエストに関するインタビュー用に、{k}任の多様なペルソナを生成してください。\n\n"
                    "ユーザリクエスト: {user_request}\n\n"
                    "各ペルソナには名前と簡単な背景を含めてください。\n"
                    "年齢、性別、職業において多様性を確保してください。"
                ),
            ]
        )
        chain = prompt | self.model
        personas = chain.invoke({"k": self.k, "user_request": user_request})

        return personas

In [13]:
# インタビュー実施
class InterviewConductor:
    def __init__(self, model: ChatOpenAI):
        self.model = model

    def run(self, user_request: str, personas: list[Persona]) -> InterviewResult:
        list_interview = []
        for p in personas:
            # 質問作成
            question = self._generate_question(user_request, p)

            # 回答作成
            answer = self._generate_answer(user_request, p, question)

            # インタビュー結果作成
            interview = Interview(persona=p, question=question, answer=answer)
            list_interview.append(interview)

        interview_result = InterviewResult(interviews = list_interview)

        return interview_result
            

    # 質問作成
    def _generate_question(self, user_request: str, persona: Persona) -> str:
        prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "あなたはユーザ要件に基づいて、適切な質問を生成する専門家です。",
                ),
                (
                    "human",
                    "以下のペルソナに関連するユーザリクエストについて、1つの質問を生成してください。\n"
                    "質問は具体的で、ペルソナの視点から重要な情報を引き出すように設計してください。\n\n"
                    "ユーザリクエスト: {user_request}\n"
                    "ペルソナ: {persona_name} - {persona_background}"
                ),
                    
            ]
        )
        chain = prompt | self.model | StrOutputParser()
        question = chain.invoke({
            "user_request": user_request,
            "persona_name": persona.name,
            "persona_background": persona.background,
        })

        return question

    # 回答作成
    def _generate_answer(self, user_request: str, persona: Persona, question: str) -> str:
        prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "あなたは以下のペルソナとして回答しています: {persona_name} - {persona_background}",
                ),
                (
                    "human",
                    "質問: {question}",
                ),    
            ]
        )
        chain = prompt | self.model | StrOutputParser()
        answer = chain.invoke({
            "persona_name": persona.name,
            "persona_background": persona.background,
            "question": question,
        })

        return answer

In [14]:
# 情報が十分に足りているか判断
class InformationEvaluator:
    def __init__(self, model: ChatOpenAI):
        self.model = model.with_structured_output(EvaluationResult)

    def run(self, user_request: str, interviews: list[Interview]) -> EvaluationResult:
        prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "あなたは包括的な要件文書を作成するための情報の十分性を評価する専門家です。",
                ),
                (
                    "human",
                    "以下のユーザリクエストとインタビュー結果に基づいて、"
                    "包括的な要件文書を作成するのに十分な情報が集まったかどうかを判断してください。\n\n"
                    "ユーザリクエスト: {user_request}\n\n"
                    "インタビュー結果: \n{interview_results}"
                ),
            ]
        )
        chain = prompt | self.model
        
        evaluation_result = chain.invoke({
            "user_request": user_request,
            "interview_results": "\n".join(
                f"ペルソナ: {i.persona.name} - {i.persona.background}\n"
                f"質問: {i.question}\n"
                f"回答: {i.answer}\n"
                for i in interviews
            )
        })

        return evaluation_result

In [15]:
# 要件定義書生成
class RequirementsDocumentGenerator:
    def __init__(self, model: ChatOpenAI):
        self.model = model

    def run(self, user_request: str, interviews: list[Interview]) -> str:
        prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "あなたは収集した情報に基づいて要件文書を作成する専門家です。",
                ),
                (
                    "human",
                    "以下のユーザリクエストと複数のペルソナからのインタビュー結果に基づいて、要件文書を作成してください\n"
                    "要件文書には以下のセクションを含めてください:\n"
                    "1. プロジェクト概要\n"
                    "2. 主要機能\n"
                    "3. 非機能要件\n"
                    "4. ターゲットユーザ\n"
                    "5. リスト区と軽減策\n\n"
                    "ユーザリクエスト: {user_request}\n\n"
                    "インタビュー結果: \n{interview_results}\n\n"
                    "要件定義書:",
                ),
            ]
        )
        chain = prompt | self.model | StrOutputParser()
        
        requirements_doc = chain.invoke({
            "user_request": user_request,
            "interview_results": "\n".join(
                f"ペルソナ: {i.persona.name} - {i.persona.background}\n"
                f"質問: {i.question}\n"
                f"回答: {i.answer}\n"
                for i in interviews
            )
        })

        return requirements_doc

## graphの定義

In [16]:
from langgraph.graph import END, StateGraph
from typing import Any

In [17]:
class DocumentationAgent:
    def __init__(self, model: ChatOpenAI, k: int = None):
        # nodeの実処理を定義したインスタンスを初期化
        self.persona_generator = PersonaGenerator(model=model, k=k)
        self.interview_conductor = InterviewConductor(model=model)
        self.information_evaluator = InformationEvaluator(model=model)
        self.requirements_generator = RequirementsDocumentGenerator(model=model)

        self.graph = self._create_graph()

    def run(self, user_request: str) -> str:
        # 初期state
        initial_state = InterviewState(user_request=user_request)

        # グラフ実行
        final_state = self.graph.invoke(initial_state)

        return final_state

    # nodeを定義
    def _generate_personas_node(self, state: InterviewState) -> dict[str, Any]:
        user_request = state.user_request
        iteration = state.iteration
    
        personas = self.persona_generator.run(user_request)
        iteration += 1

        return {
            "personas": personas.personas,
            "iteration": iteration,
        }

    def _conduct_interview_node(self, state: InterviewState) -> dict[str, Any]:
        user_request = state.user_request
        personas = state.personas[-5:] # 本来はkで動的に値を変えるべき
    
        interview_rusult = self.interview_conductor.run(user_request, personas)

        return {
            "interviews": interview_rusult.interviews
        }

    def _evaluate_information_node(self, state: InterviewState) -> dict[str, Any]:
        user_request = state.user_request
        interviews = state.interviews[-5:] # 本来はkで動的に値を変えるべき
    
        evaluation_result = self.information_evaluator.run(user_request, interviews)

        return {
            "is_information_sufficient": evaluation_result.is_sufficient,
            "evaluation_reason": evaluation_result.reason,
        }

    def _generate_requirements_node(self, state: InterviewState) -> dict[str, Any]:
        user_request = state.user_request
        interviews = state.interviews[-5:] # 本来はkで動的に値を変えるべき
    
        requirements_doc: str = self.requirements_generator.run(user_request, interviews)

        return {
            "requirements_doc": requirements_doc,
        }
    
    # graphを定義
    def _create_graph(self) -> StateGraph:

        # graphの初期化
        workflow = StateGraph(InterviewState)

        # nodeを追加
        workflow.add_node("generate_personas", self._generate_personas_node)
        workflow.add_node("conduct_interview", self._conduct_interview_node)
        workflow.add_node("evaluate_information", self._evaluate_information_node)
        workflow.add_node("generate_requirements", self._generate_requirements_node)
        
        # entry nodeを設定
        workflow.set_entry_point("generate_personas")

        #　edgeを設定
        workflow.add_edge("generate_personas", "conduct_interview")
        workflow.add_edge("conduct_interview", "evaluate_information")
        workflow.add_edge("evaluate_information", "generate_requirements")
        workflow.add_edge("generate_requirements", END)

        compiled = workflow.compile()

        return compiled

## 実行

In [18]:
model = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
query = "健康管理をするアプリを開発したい。"
dg = DocumentationAgent(model=model, k=5)
state = dg.run(query)

In [19]:
state

{'user_request': '健康管理をするアプリを開発したい。',
 'personas': [Persona(name='佐藤健一', background='35歳の男性。IT企業でエンジニアとして働いている。健康に気を使い始めたが、忙しい仕事の合間に運動や食事管理が難しいと感じている。'),
  Persona(name='山田美咲', background='28歳の女性。フリーランスのデザイナーで、在宅勤務が多い。健康的なライフスタイルを目指しているが、ストレスや不規則な生活が影響している。'),
  Persona(name='鈴木太郎', background='50歳の男性。中小企業の経営者。最近健康診断で高血圧と診断され、生活習慣を見直す必要があると感じている。忙しい日々の中で簡単に健康管理ができる方法を探している。'),
  Persona(name='田中花子', background='22歳の女性。大学生で、友人と一緒にフィットネスに通っている。健康に興味があり、アプリを使ってトレーニングや食事の記録をしたいと思っている。'),
  Persona(name='中村健二', background='45歳の男性。公務員で、家族を持つ。健康維持のために運動を始めたいが、時間が取れずにいる。家族全員で使える健康管理アプリを求めている。')],
 'interviews': [Interview(persona=Persona(name='佐藤健一', background='35歳の男性。IT企業でエンジニアとして働いている。健康に気を使い始めたが、忙しい仕事の合間に運動や食事管理が難しいと感じている。'), question='佐藤健一さんのような忙しいエンジニアにとって、健康管理アプリに求める最も重要な機能は何ですか？例えば、運動のスケジュール管理や食事の簡単な記録機能など、具体的にどのようなサポートがあれば日常生活に取り入れやすいと感じますか？', answer='忙しいエンジニアとして、健康管理アプリに求める最も重要な機能はいくつかありますが、特に以下の点が重要だと感じます。\n\n1. **簡単な食事記録機能**: 食事を記録するのが手間にならないように、バーコードスキャンやお気に入りの食事を登録できる機能があると便利です。忙

In [20]:
from IPython.display import display, Markdown

In [21]:
display(Markdown(state["requirements_doc"]))

# 要件定義書

## 1. プロジェクト概要
本プロジェクトは、忙しい日常の中で健康管理を行うためのアプリケーションを開発することを目的としています。ユーザーが簡単に食事や運動を記録し、健康状態を管理できる機能を提供することで、健康的なライフスタイルを促進します。ターゲットユーザーは、忙しいビジネスパーソン、在宅勤務者、家族全員で利用できる機能を求めるユーザーなど多岐にわたります。

## 2. 主要機能
1. **食事記録機能**
   - バーコードスキャンによる簡単な食事記録
   - お気に入りの食事登録機能
   - 栄養分析機能

2. **運動管理機能**
   - 運動スケジュール管理
   - 短時間でできるエクササイズの提案
   - リマインダー機能

3. **ストレス管理機能**
   - ストレストラッキング機能
   - マインドフルネスや瞑想のガイド

4. **進捗の可視化**
   - 健康状態や運動の進捗をグラフやチャートで表示

5. **コミュニティ機能**
   - 他のユーザーとの情報交換や励まし合いの場を提供

6. **家族アカウントの共有**
   - 家族全員がそれぞれのアカウントを持ち、進捗や目標を共有できる機能

7. **健康相談窓口**
   - 専門家に気軽に相談できるオンライン窓口

8. **フィードバック機能**
   - 健康データに基づく定期的なフィードバックやアドバイス

## 3. 非機能要件
1. **ユーザビリティ**
   - 直感的で使いやすいインターフェース
   - スマートフォンおよびタブレットに対応

2. **パフォーマンス**
   - アプリの起動時間は3秒以内
   - データの同期はリアルタイムで行う

3. **セキュリティ**
   - ユーザーデータの暗号化
   - プライバシーポリシーの遵守

4. **互換性**
   - iOSおよびAndroidプラットフォームに対応

5. **スケーラビリティ**
   - ユーザー数の増加に対応できるアーキテクチャ

## 4. ターゲットユーザ
- **佐藤健一**: 35歳男性、IT企業のエンジニア。忙しい日常の中で健康管理を行いたい。
- **山田美咲**: 28歳女性、フリーランスのデザイナー。ストレス管理や生活リズムの改善を求めている。
- **鈴木太郎**: 50歳男性、中小企業の経営者。高血圧の改善を目指している。
- **田中花子**: 22歳女性、大学生。友人と一緒にフィットネスを楽しみたい。
- **中村健二**: 45歳男性、公務員。家族全員で健康管理を行いたい。

## 5. リスクと軽減策
| リスク | 軽減策 |
|--------|--------|
| ユーザーの利用継続率が低い | コミュニティ機能やリマインダー機能を強化し、ユーザーのモチベーションを維持 |
| データのセキュリティリスク | ユーザーデータの暗号化とプライバシーポリシーの徹底 |
| 技術的な問題によるパフォーマンス低下 | 定期的なテストとメンテナンスを実施し、パフォーマンスを監視 |
| ユーザーのニーズの変化 | 定期的なユーザーインタビューやフィードバックを通じて機能改善を行う |

以上が健康管理アプリの要件定義書です。このアプリがユーザーの健康管理をサポートし、より健康的なライフスタイルを実現する手助けとなることを目指します。